In [ ]:
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl

In [ ]:
cd '/content/drive/MyDrive/NLP/NLP project/legal_NER-main'

In [ ]:
!pip install -r requirements.txt

In [3]:
import pandas as pd

with open('/content/NER_DEV_JUDGEMENT.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

df.to_csv('csvfile.csv', encoding='utf-8', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df

,id,annotations,data,meta
0,03f3901e95ed493b866bd7807f623bc0,"[{'result': [{'value': {'start': 10, 'end': 22...","{'text': 'True, our Constitution has no 'due p...",{'source': 'criminal_punjab-haryana_high_court...
1,b0311cba3aac4d909eec6e156c059617,"[{'result': [{'value': {'start': 55, 'end': 65...",{'text': '(See Principles of Statutory Interpr...,{'source': 'financial_rajasthan_high_court jud...
2,0215658407ec479fa4e0e73ca0bf98b6,"[{'result': [{'value': {'start': 101, 'end': 1...","{'text': 'Their Lordships have said -- ""It is...",{'source': 'financial_rajasthan_high_court jud...
3,cf44f4898b67493390f1a8c8c7dfc0cc,"[{'result': [{'value': {'start': 252, 'end': 2...","{'text': 'In para 13 of the plaint, it has bee...",{'source': 'criminal_districtcourts judgement ...
4,f16b5b960a854b18bf171b4cff9b5b11,"[{'result': [{'value': {'start': 169, 'end': 1...",{'text': 'Counsel for appellants contended tha...,{'source': 'tax_telangana_high_court judgement...
...,...,...,...,...
944,ffcb5e8de37c43028bca42ba83852d16,"[{'result': [{'value': {'start': 12, 'end': 23...","{'text': 'Apparently, Channaraddi set up his d...",{'source': 'tax_karnataka_high_court judgement...
945,e848804c6a014d21afeba933b1dcc73a,"[{'result': [{'value': {'start': 89, 'end': 11...",{'text': 'After the dismissal of the petition ...,{'source': 'civil_andhra_high_court judgement ...
946,47f8e3fac6924cfaaa8f9a7194aee871,"[{'result': [{'value': {'start': 3, 'end': 13,...","{'text': 'On 12.07.2018, a letter was received...",{'source': 'industrial&labour_supremecourts ju...
947,5eeb09ca4608463cb64b0d8ee72a4a21,[{'result': []}],{'text': 'The date on which the measurements w...,{'source': 'constitution_karnataka_high_court ...


In [5]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Text Preprocessing

### Preproceesing annotations start

In [6]:
annotations_column = df['annotations']

In [7]:
annotations_column =  annotations_column.to_numpy()

In [8]:
annotation_start = []
annotation_end = []


for sent in annotations_column:
  sent_start = []
  sent_end = []
  sent_text = []
  sent_label = []
  for sent_res in sent:
    for res_value in sent_res['result']:
      sent_start.append(res_value['value']['start'])

    for res_value in sent_res['result']:
      sent_end.append(res_value['value']['end'])
    


  annotation_start.append(sent_start)
  annotation_end.append(sent_end)


In [9]:
annotation_start[0]

[10, 108, 160]

In [10]:
len(annotation_end)

949

In [11]:
annotation_text = []
annotation_label = []
for sent in annotations_column:
    sent_text = []
    sent_label = []
    for sent_res in sent:

        for res_value in sent_res['result']:
          sent_text.append(res_value['value']['text'])
        
        for res_value in sent_res['result']:
          sent_label.append(res_value['value']['labels']) 

    annotation_text.append(sent_text)
    annotation_label.append(sent_label)

In [12]:
annotation_text[0]

['Constitution',
 'R.C. Cooper v. Union of India, (1970) 1 SCC 248',
 'Maneka Gandhi v. Union of India, (1978) 1 SCC 248']

In [13]:
annotation_label[0]

[['STATUTE'], ['PRECEDENT'], ['PRECEDENT']]

In [14]:
column_names = ['start', 'end' , 'text' , 'labels']

final_df = pd.DataFrame()
final_df['start'] = annotation_start
final_df['end'] = annotation_end
final_df['text'] = annotation_text
final_df['labels'] = annotation_label

final_df.head()

,start,end,text,labels
0,"[10, 108, 160]","[22, 155, 209]","[Constitution, R.C. Cooper v. Union of India, ...","[[STATUTE], [PRECEDENT], [PRECEDENT]]"
1,[55],[65],[G.P. Singh],[[JUDGE]]
2,"[101, 134]","[108, 140]","[England, Heydon]","[[GPE], [OTHER_PERSON]]"
3,"[252, 313]","[262, 318]","[29/12/2004, Delhi]","[[DATE], [GPE]]"
4,"[169, 275, 294]","[175, 286, 326]","[Rule 2, Section 172, Hyderabad Land Revenue A...","[[PROVISION], [PROVISION], [STATUTE]]"


In [15]:
for a,b,c,d in zip(annotation_start[:2], annotation_end[:2],annotation_text[:2],annotation_label[:2]):
  print(a,b,c,d)

[10, 108, 160] [22, 155, 209] ['Constitution', 'R.C. Cooper v. Union of India, (1970) 1 SCC 248', 'Maneka Gandhi v. Union of India, (1978) 1 SCC 248'] [['STATUTE'], ['PRECEDENT'], ['PRECEDENT']]
[55] [65] ['G.P. Singh'] [['JUDGE']]


In [16]:
zip(annotation_start[0], annotation_end[0])

In [17]:
pwd

'/content'

In [18]:
final_df.to_csv('Processed_DataFrame_Dev')

## IOB tagging

In [19]:
import pandas as pd

df_ = pd.read_csv('/content/Processed_DataFrame_Dev')
df_.head()

,Unnamed: 0,start,end,text,labels
0,0,"[10, 108, 160]","[22, 155, 209]","['Constitution', 'R.C. Cooper v. Union of Indi...","[['STATUTE'], ['PRECEDENT'], ['PRECEDENT']]"
1,1,[55],[65],['G.P. Singh'],[['JUDGE']]
2,2,"[101, 134]","[108, 140]","['England', 'Heydon']","[['GPE'], ['OTHER_PERSON']]"
3,3,"[252, 313]","[262, 318]","['29/12/2004', 'Delhi']","[['DATE'], ['GPE']]"
4,4,"[169, 275, 294]","[175, 286, 326]","['Rule 2', 'Section 172', 'Hyderabad Land Reve...","[['PROVISION'], ['PROVISION'], ['STATUTE']]"


In [20]:
text = df.iloc[[0]]['data']

In [21]:
text[0]['text']

"True, our Constitution has no 'due process' clause or the VIII Amendment; but, in this branch of law, after R.C. Cooper v. Union of India, (1970) 1 SCC 248 and Maneka Gandhi v. Union of India, (1978) 1 SCC 248, the consequence is the same."

In [22]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(text[0]['text'])

for d in doc:
  print(d.text)

True
,
our
Constitution
has
no
'
due
process
'
clause
or
the
VIII
Amendment
;
but
,
in
this
branch
of
law
,
after
R.C.
Cooper
v.
Union
of
India
,
(
1970
)
1
SCC
248
and
Maneka
Gandhi
v.
Union
of
India
,
(
1978
)
1
SCC
248
,
the
consequence
is
the
same
.


In [23]:
import re
t = re.sub('\n', " ", text[0]['text'])


In [24]:
" ".join(t.split())

"True, our Constitution has no 'due process' clause or the VIII Amendment; but, in this branch of law, after R.C. Cooper v. Union of India, (1970) 1 SCC 248 and Maneka Gandhi v. Union of India, (1978) 1 SCC 248, the consequence is the same."

In [25]:
df['data']

0      {'text': 'True, our Constitution has no 'due p...
1      {'text': '(See Principles of Statutory Interpr...
2      {'text': 'Their Lordships have said --  "It is...
3      {'text': 'In para 13 of the plaint, it has bee...
4      {'text': 'Counsel for appellants contended tha...
                             ...                        
944    {'text': 'Apparently, Channaraddi set up his d...
945    {'text': 'After the dismissal of the petition ...
946    {'text': 'On 12.07.2018, a letter was received...
947    {'text': 'The date on which the measurements w...
948    {'text': 'The lower back end was 5 cm behind r...
Name: data, Length: 949, dtype: object

In [26]:
def preprocess(sent):
  sent = re.sub('\n', " ", sent)
  sent = sent.replace("...", " ")
  sent = " ".join(sent.split())

  return sent


clean_text = []
for i in df['data']:
  clean_text.append(preprocess(i['text']))

In [27]:
len(clean_text)

949

In [28]:
df_['clean_text'] = clean_text

In [29]:
df_.head()

,Unnamed: 0,start,end,text,labels,clean_text
0,0,"[10, 108, 160]","[22, 155, 209]","['Constitution', 'R.C. Cooper v. Union of Indi...","[['STATUTE'], ['PRECEDENT'], ['PRECEDENT']]","True, our Constitution has no 'due process' cl..."
1,1,[55],[65],['G.P. Singh'],[['JUDGE']],(See Principles of Statutory Interpretation by...
2,2,"[101, 134]","[108, 140]","['England', 'Heydon']","[['GPE'], ['OTHER_PERSON']]","Their Lordships have said -- ""It is a sound ru..."
3,3,"[252, 313]","[262, 318]","['29/12/2004', 'Delhi']","[['DATE'], ['GPE']]","In para 13 of the plaint, it has been further ..."
4,4,"[169, 275, 294]","[175, 286, 326]","['Rule 2', 'Section 172', 'Hyderabad Land Reve...","[['PROVISION'], ['PROVISION'], ['STATUTE']]",Counsel for appellants contended that who is t...


In [30]:
s = df_.iloc[0]['clean_text']

for i in range(7,40):
  print(s[i])

u
r
 
C
o
n
s
t
i
t
u
t
i
o
n
 
h
a
s
 
n
o
 
'
d
u
e
 
p
r
o
c
e


In [31]:
df_.iloc[1]['clean_text']

import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(df_.iloc[0]['clean_text'])

print(doc[0])

True


In [32]:
span = doc.char_span(73,100,label = "COURT")
span

## New solution

In [33]:
def sentence_cleaner(sentence):
    '''
    replacing the unwanted charachters
    '''
    reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
    return " ".join(reg.findall(sentence.replace('\n', ' '))).replace(" / ", '/')

In [34]:
def sentence_cleaner_v1(sentence):
    '''
    replacing the unwanted charachters
    '''
    sent_1 = " ".join(sentence.replace("\n" , " ").replace("..." , " ").replace("-", " ").replace("," , " ").replace(":" , " ").replace("=",  " ").split())
    doc = nlp(sent_1)
    doc_list = []
    for i in doc:
      doc_list.append(str(i))
    return doc_list

In [35]:
def sentence_tagging(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,temp_len)
    if temp_len > 1 :
        for i in range(temp_len):
            #print(i, temp_len - 1)
            if i == 0 :
              word2ent[temp_sent[i]] =  "B-" + ann['value']['labels'][0]
            
            elif i == (temp_len - 1):
              word2ent[temp_sent[i]] =  "E-" + ann['value']['labels'][0]

            else :
              word2ent[temp_sent[i]] =  "I-" + ann['value']['labels'][0]

    else :
        word2ent[temp_sent[0]] = "B-" + ann['value']['labels'][0]

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys() and word + ',' not in word2ent.keys() :
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
        tag_list.append(word2ent[word])
      
      else :
          tag_list.append(word2ent[word])
  
  
  print(word2ent )
  return tag_list
      

In [36]:
def sentence_tagging_v1(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
      tag_list.append(word2ent[word])
      
  
  
  #print(word2ent )
  return tag_list
      

In [37]:
def sentence_tagging_v2(sent, annotation):

  """
  current using word
  """

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = sentence_cleaner_v1(sent[ ann['value']['start']: ann['value']['end']])
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner_v1(sent)
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        tag = "O"
      else :
        if len(word2ent[word]) != 0 :
          tag = word2ent[word].pop(0)
        else :
          tag = "O"

      #print(word,tag)   
      tag_list.append(str(tag))
      
  
  
  #print(word2ent )
  return tag_list
      

In [38]:
IOBE_tagging = []
i = 0
for sent, anno in zip( df['data'].to_list(), df['annotations'].to_list()):
    IOBE_tagging.append(sentence_tagging_v2(sent,anno))
    
  

In [39]:
IOBE_tagging[2]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-GPE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-OTHER_PERSON',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [40]:
final_df['tags'] = IOBE_tagging
final_df.head()

,start,end,text,labels,tags
0,"[10, 108, 160]","[22, 155, 209]","[Constitution, R.C. Cooper v. Union of India, ...","[[STATUTE], [PRECEDENT], [PRECEDENT]]","[O, O, B-STATUTE, O, O, O, O, O, O, O, O, O, O..."
1,[55],[65],[G.P. Singh],[[JUDGE]],"[O, O, O, O, O, O, O, O, B-JUDGE, E-JUDGE, O, ..."
2,"[101, 134]","[108, 140]","[England, Heydon]","[[GPE], [OTHER_PERSON]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[252, 313]","[262, 318]","[29/12/2004, Delhi]","[[DATE], [GPE]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[169, 275, 294]","[175, 286, 326]","[Rule 2, Section 172, Hyderabad Land Revenue A...","[[PROVISION], [PROVISION], [STATUTE]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [41]:
sent_1 = df['data'].to_list()[2]['text']
sent_1

'Their Lordships have said --  "It is a sound rule of construction of a statute firmly established in England as far back as 1584 when Heydon\'s case was decided that --"......'

In [ ]:
# tag_2 = final_df['tags'].to_list()[77]
# tag_2

In [42]:
# [\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]

reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
" ".join(reg.findall(sent_1.replace('\n', ' '))).replace(" / ", '/')

"Their Lordships have said It is a sound rule of construction of a statute firmly established in England as far back as 1584 when Heydon's case was decided that"

In [43]:
" ".join(sent_1.replace(" "," ").split())

'Their Lordships have said -- "It is a sound rule of construction of a statute firmly established in England as far back as 1584 when Heydon\'s case was decided that --"......'

In [44]:
word2count = {}

for sent in df['data'].to_list():
    for word in sent['text'].split():
        if word not in word2count:
          word2count[word] = 0
        else:
          word2count[word] += 1

In [45]:
for key in word2count.keys():
  if re.search(r'\bM+\b', key):
    print(key)

M.D.
Mr.L.M.Menezes
M/s
M.A.C.M.A.
M/s.Maheswari
M.L.
M.P.;
M/s.
Crl.M.C.2719
M.P.
MHC(M)
MM
M.P
M.
Smt.M.Printer
A.M.
M.Philip
O.M.
M.C.O.P.No.2224
P.M
A.M
P.M.,
M.Anil
M.A.C.M.A.No.685
M.A.C.M.A.No.1233
(O&M)
M.Thombra
M/S.Evershine
M.R.
M


## Using Tokenizer

In [46]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

In [47]:
sent_1 = " ".join(sent_1.replace("\n" , " ").replace("..." , " ").replace("-", "").split())
doc = nlp(sent_1)
doc_list = []
for i in doc:
  doc_list.append(str(i))
doc_list

['Their',
 'Lordships',
 'have',
 'said',
 '"',
 'It',
 'is',
 'a',
 'sound',
 'rule',
 'of',
 'construction',
 'of',
 'a',
 'statute',
 'firmly',
 'established',
 'in',
 'England',
 'as',
 'far',
 'back',
 'as',
 '1584',
 'when',
 'Heydon',
 "'s",
 'case',
 'was',
 'decided',
 'that',
 '"']

# Clean the sentences

In [48]:
clean_sentences = []

for sent in df['data'].to_list():
    clean_sentences.append(sentence_cleaner_v1(sent['text']))



In [49]:
final_df['clean_sent'] = clean_sentences
final_df.head()

,start,end,text,labels,tags,clean_sent
0,"[10, 108, 160]","[22, 155, 209]","[Constitution, R.C. Cooper v. Union of India, ...","[[STATUTE], [PRECEDENT], [PRECEDENT]]","[O, O, B-STATUTE, O, O, O, O, O, O, O, O, O, O...","[True, our, Constitution, has, no, ', due, pro..."
1,[55],[65],[G.P. Singh],[[JUDGE]],"[O, O, O, O, O, O, O, O, B-JUDGE, E-JUDGE, O, ...","[(, See, Principles, of, Statutory, Interpreta..."
2,"[101, 134]","[108, 140]","[England, Heydon]","[[GPE], [OTHER_PERSON]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Their, Lordships, have, said, "", It, is, a, s..."
3,"[252, 313]","[262, 318]","[29/12/2004, Delhi]","[[DATE], [GPE]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[In, para, 13, of, the, plaint, it, has, been,..."
4,"[169, 275, 294]","[175, 286, 326]","[Rule 2, Section 172, Hyderabad Land Revenue A...","[[PROVISION], [PROVISION], [STATUTE]]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Counsel, for, appellants, contended, that, wh..."


## Word2idx

In [50]:
def get_dict_map(df , token_or_tag):
    """
    converts word to index

    [Need changes !!!]
    """
    tok2idx = {}
    idx2tok = {}

    vocab = []

    if token_or_tag == 'token':
        for sent in df['clean_sent'].to_list():
          vocab.extend( list(set(sent)) )

        vocab = set(vocab)

    else :
        for sent in df['tags'].to_list():
          vocab.extend( list(set(sent)) )
        
        vocab = set(vocab)


    idx2tok = {idx:tok for idx , tok in enumerate(vocab)}
    tok2idx = {tok:idx for idx , tok in enumerate(vocab)}

    return tok2idx, idx2tok



word2idx, idx2word = get_dict_map(final_df, 'token')
# tag2idx, idx2tag = get_dict_map(final_df, 'tag')


In [51]:
import json
 
# Opening JSON file
with open('tag2idx_Judgement') as tag2idx_file:
    tag2idx = json.load(tag2idx_file)

In [52]:
import json
 
# Opening JSON file
with open('idx2tag_Judgement') as idx2tag_file:
    idx2tag = json.load(idx2tag_file)
     

In [53]:
tag2idx

{'E-RESPONDENT': 0,
 'B-RESPONDENT': 1,
 'O': 2,
 'B-GPE': 3,
 'I-OTHER_PERSON': 4,
 'B-PROVISION': 5,
 'E-ORG': 6,
 'E-PROVISION': 7,
 'E-GPE': 8,
 'B-STATUTE': 9,
 'B-JUDGE': 10,
 'B-ORG': 11,
 'E-JUDGE': 12,
 'B-OTHER_PERSON': 13,
 'I-WITNESS': 14,
 'I-DATE': 15,
 'I-PROVISION': 16,
 'I-COURT': 17,
 'I-CASE_NUMBER': 18,
 'B-PETITIONER': 19,
 'I-PRECEDENT': 20,
 'I-JUDGE': 21,
 'E-CASE_NUMBER': 22,
 'I-ORG': 23,
 'E-OTHER_PERSON': 24,
 'B-WITNESS': 25,
 'I-PETITIONER': 26,
 'E-WITNESS': 27,
 'E-PRECEDENT': 28,
 'B-CASE_NUMBER': 29,
 'B-PRECEDENT': 30,
 'I-GPE': 31,
 'I-RESPONDENT': 32,
 'B-DATE': 33,
 'E-DATE': 34,
 'I-STATUTE': 35,
 'B-COURT': 36,
 'E-COURT': 37,
 'E-PETITIONER': 38,
 'E-STATUTE': 39}

In [54]:
idx2tag

{'0': 'E-RESPONDENT',
 '1': 'B-RESPONDENT',
 '2': 'O',
 '3': 'B-GPE',
 '4': 'I-OTHER_PERSON',
 '5': 'B-PROVISION',
 '6': 'E-ORG',
 '7': 'E-PROVISION',
 '8': 'E-GPE',
 '9': 'B-STATUTE',
 '10': 'B-JUDGE',
 '11': 'B-ORG',
 '12': 'E-JUDGE',
 '13': 'B-OTHER_PERSON',
 '14': 'I-WITNESS',
 '15': 'I-DATE',
 '16': 'I-PROVISION',
 '17': 'I-COURT',
 '18': 'I-CASE_NUMBER',
 '19': 'B-PETITIONER',
 '20': 'I-PRECEDENT',
 '21': 'I-JUDGE',
 '22': 'E-CASE_NUMBER',
 '23': 'I-ORG',
 '24': 'E-OTHER_PERSON',
 '25': 'B-WITNESS',
 '26': 'I-PETITIONER',
 '27': 'E-WITNESS',
 '28': 'E-PRECEDENT',
 '29': 'B-CASE_NUMBER',
 '30': 'B-PRECEDENT',
 '31': 'I-GPE',
 '32': 'I-RESPONDENT',
 '33': 'B-DATE',
 '34': 'E-DATE',
 '35': 'I-STATUTE',
 '36': 'B-COURT',
 '37': 'E-COURT',
 '38': 'E-PETITIONER',
 '39': 'E-STATUTE'}

In [55]:
"""
Padding and converting from word sequence to index sequence

"""

max_len = max([len(sent) for sent in final_df['clean_sent'].tolist()])
pad_idx = len(word2idx)
tag_pad_idx = tag2idx['O']
word_idx_sequence = []
tag_idx_sequence = []
pad = "PAD"

for sent in final_df['clean_sent'].to_list():
    #print(sent)
    word_idx_sequence.append([word2idx[word] for word in sent] + [pad_idx for _ in range(max_len - len(sent))])


for tag_seq in final_df['tags'].to_list():
    tag_idx_sequence.append([tag2idx[tag] for tag in tag_seq] + [tag_pad_idx for _ in range(max_len - len(tag_seq))])


In [56]:
final_df['word_idx'] = word_idx_sequence
final_df['tag_idx'] = tag_idx_sequence



In [57]:
len(tag2idx)

40

In [58]:
tag2idx

{'E-RESPONDENT': 0,
 'B-RESPONDENT': 1,
 'O': 2,
 'B-GPE': 3,
 'I-OTHER_PERSON': 4,
 'B-PROVISION': 5,
 'E-ORG': 6,
 'E-PROVISION': 7,
 'E-GPE': 8,
 'B-STATUTE': 9,
 'B-JUDGE': 10,
 'B-ORG': 11,
 'E-JUDGE': 12,
 'B-OTHER_PERSON': 13,
 'I-WITNESS': 14,
 'I-DATE': 15,
 'I-PROVISION': 16,
 'I-COURT': 17,
 'I-CASE_NUMBER': 18,
 'B-PETITIONER': 19,
 'I-PRECEDENT': 20,
 'I-JUDGE': 21,
 'E-CASE_NUMBER': 22,
 'I-ORG': 23,
 'E-OTHER_PERSON': 24,
 'B-WITNESS': 25,
 'I-PETITIONER': 26,
 'E-WITNESS': 27,
 'E-PRECEDENT': 28,
 'B-CASE_NUMBER': 29,
 'B-PRECEDENT': 30,
 'I-GPE': 31,
 'I-RESPONDENT': 32,
 'B-DATE': 33,
 'E-DATE': 34,
 'I-STATUTE': 35,
 'B-COURT': 36,
 'E-COURT': 37,
 'E-PETITIONER': 38,
 'E-STATUTE': 39}

In [59]:
word2idx

{'provide': 0,
 'Guwahati': 1,
 'Breeders': 2,
 'stressed': 3,
 'pleadings': 4,
 'Vishal': 5,
 '072': 6,
 'shifts': 7,
 'namely': 8,
 'office': 9,
 'integra': 10,
 'Magsha': 11,
 'incapable': 12,
 'significance': 13,
 'law': 14,
 'clarified': 15,
 '9': 16,
 '9360': 17,
 'survive': 18,
 'PW13': 19,
 'From': 20,
 'consecutive': 21,
 'Marwah': 22,
 'quality': 23,
 'tile': 24,
 'makers': 25,
 'bringing': 26,
 'level': 27,
 'Reshma': 28,
 'know': 29,
 'US': 30,
 'Constable': 31,
 'PW2': 32,
 'produce': 33,
 'recruit': 34,
 'Reserve': 35,
 'words': 36,
 'effectively': 37,
 'nowhere': 38,
 'achieve': 39,
 'No.12': 40,
 '6520': 41,
 'held': 42,
 'misconduct': 43,
 'type': 44,
 'retrospective': 45,
 'Versus': 46,
 'Mihir': 47,
 'Bukkarayasamudram': 48,
 'Mukesh': 49,
 'proviso': 50,
 'venue': 51,
 'hulla': 52,
 'Generation': 53,
 'devise': 54,
 'draft': 55,
 'ratio': 56,
 'Portfolio': 57,
 'Konavalov': 58,
 'http': 59,
 'Krishna': 60,
 'southern': 61,
 'never': 62,
 'PW\xad5': 63,
 'Aligarh': 6

In [60]:
final_df.to_csv("Final_DataFrame_Dev_Judgement")